# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import re
import nltk

from sqlalchemy import create_engine

# imports from nltk-submodules
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# imports from sklearn-submodules
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis¶
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# downloads from nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True);

In [46]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('data/DisasterResponse.db', engine)
X = df[['message']].values[:, 0]
Y = df[[column for column in df.columns if column != 'message']].values

### 2. Write a tokenization function to process your text data

In [47]:
def tokenize(text):
    
    # define stop_words and lemmatizer 
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [48]:
pipeline = Pipeline([
    ('vectr', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('rfclf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced', n_estimators=20)))
], verbose=True)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [51]:
pipeline.fit(X_train, Y_train);

[Pipeline] ............. (step 1 of 3) Processing vectr, total=   8.4s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............. (step 3 of 3) Processing rfclf, total= 1.1min


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [52]:
target_names = [column for column in df.columns if column != 'message']

Y_pred = pipeline.predict(X_train)
print("Training scores:")
print(classification_report(Y_train, Y_pred, target_names=target_names))

Y_pred = pipeline.predict(X_test)
print("Testing scores:")
print(classification_report(Y_test, Y_pred, target_names=target_names))

Training scores:
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00     14966
               request       0.99      0.97      0.98      3376
                 offer       0.97      0.81      0.89        91
           aid_related       1.00      0.99      0.99      8161
          medical_help       1.00      0.95      0.97      1568
      medical_products       1.00      0.93      0.96      1009
     search_and_rescue       0.99      0.87      0.93       546
              security       0.99      0.87      0.93       359
              military       0.99      0.95      0.97       636
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.98      0.99      1229
                  food       1.00      0.98      0.99      2205
               shelter       1.00      0.97      0.99      1699
              clothing       1.00      0.95      0.97       303
                 money

/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/desaster_response_pipeline/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

Testing scores:
                        precision    recall  f1-score   support

               related       0.86      0.91      0.88      4940
               request       0.76      0.51      0.61      1098
                 offer       0.00      0.00      0.00        27
           aid_related       0.74      0.67      0.70      2699
          medical_help       0.59      0.08      0.14       516
      medical_products       0.71      0.10      0.17       304
     search_and_rescue       0.57      0.02      0.04       178
              security       0.00      0.00      0.00       112
              military       0.50      0.04      0.07       224
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.36      0.50       443
                  food       0.83      0.57      0.67       718
               shelter       0.85      0.28      0.42       615
              clothing       0.57      0.04      0.07       102
                 money 

### 6. Improve your model
Use grid search to find better parameters.

In [53]:
for interesting_param in [param for param in pipeline.get_params(deep=True).keys() if param[0:7] == "rfclf__"]:
    print(interesting_param+":", pipeline.get_params()[interesting_param])

rfclf__estimator__bootstrap: True
rfclf__estimator__class_weight: balanced
rfclf__estimator__criterion: gini
rfclf__estimator__max_depth: None
rfclf__estimator__max_features: auto
rfclf__estimator__max_leaf_nodes: None
rfclf__estimator__min_impurity_decrease: 0.0
rfclf__estimator__min_impurity_split: None
rfclf__estimator__min_samples_leaf: 1
rfclf__estimator__min_samples_split: 2
rfclf__estimator__min_weight_fraction_leaf: 0.0
rfclf__estimator__n_estimators: 20
rfclf__estimator__n_jobs: None
rfclf__estimator__oob_score: False
rfclf__estimator__random_state: None
rfclf__estimator__verbose: 0
rfclf__estimator__warm_start: False
rfclf__estimator: RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_le

In [54]:
parameters = {
    'rfclf__estimator__max_features':['auto', None],
    'rfclf__estimator__n_estimators':[10, 20]
}

cv = GridSearchCV(pipeline, parameters, scoring='f1_weighted', cv=2, verbose=3, n_jobs=-1)

In [ ]:
cv.fit(X_train, Y_train);

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   51.3s remaining:  2.6min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  4.1min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  7.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  7.3min finished


[Pipeline] ............. (step 1 of 3) Processing vectr, total=   8.5s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.best_params_

In [ ]:
target_names = [column for column in df.columns if column != 'message']

Y_pred = cv.predict(X_train)
print("Training scores:")
print(classification_report(Y_train, Y_pred, target_names=target_names))

Y_pred = cv.predict(X_test)
print("Testing scores:")
print(classification_report(Y_test, Y_pred, target_names=target_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vectr', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('rfclf', MultiOutputClassifier(LinearDiscriminantAnalysis()))
], verbose=True)

In [ ]:
pipeline.fit(X_train, Y_train);

In [ ]:
target_names = [column for column in df.columns if column != 'message']

Y_pred = pipeline.predict(X_train)
print("Training scores:")
print(classification_report(Y_train, Y_pred, target_names=target_names))

Y_pred = pipeline.predict(X_test)
print("Testing scores:")
print(classification_report(Y_test, Y_pred, target_names=target_names))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.